In [1]:

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import os


2025-10-24 09:42:34.985029: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-24 09:42:35.024910: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-24 09:42:36.213903: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:

# Define constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10  # You might want to increase this for better accuracy
NUM_CLASSES = 6 # angry, Happy, Normal, relaxed, sad, Scared
DATASET_PATH = "/home/qod120/Documents/project/2nd_ai_web_project/ai_model/Cat Emotions.v2i.folder"
TRAIN_DIR = os.path.join(DATASET_PATH, "train")

MODEL_SAVE_PATH = "/home/qod120/Documents/project/2nd_ai_web_project/ai_model/cat_emotion_mobilenet_v2.h5"

# Create an ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1 # 10% for validation
)

# Create the generators
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical', # For one-hot encoded labels
    subset='training', # Set as training data
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical', # For one-hot encoded labels
    subset='validation', # Set as validation data
    shuffle=False
)

# Load MobileNetV2 pre-trained on ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

# Add custom top layers for single-label classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(NUM_CLASSES, activation='softmax')(x) # Softmax for single-label classification

# Create the new model
model = Model(inputs=base_model.input, outputs=x)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy', # Use categorical_crossentropy for single-label classification
              metrics=['accuracy'])

# Train the model
print("Starting model training...")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    verbose=1
)
print("Model training finished.")

# Save the trained model
model.save(MODEL_SAVE_PATH)
print(f"Model saved to {MODEL_SAVE_PATH}")


Found 957 images belonging to 6 classes.
Found 104 images belonging to 6 classes.


2025-10-24 09:42:37.262578: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Starting model training...
Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 24s 733ms/step - accuracy: 0.6832 - loss: 1.1020 - val_accuracy: 0.7188 - val_loss: 0.8229
Epoch 2/10
 1/29 ━━━━━━━━━━━━━━━━━━━━ 9s 340ms/step - accuracy: 0.7188 - loss: 0.9447

/home/qod120/anaconda3/lib/python3.13/site-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.7188 - loss: 0.9447 - val_accuracy: 0.7188 - val_loss: 0.8374
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 686ms/step - accuracy: 0.7405 - loss: 0.8020 - val_accuracy: 0.7604 - val_loss: 0.7298
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - accuracy: 0.7188 - loss: 0.6531 - val_accuracy: 0.7500 - val_loss: 0.7111
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 653ms/step - accuracy: 0.7741 - loss: 0.6873 - val_accuracy: 0.7917 - val_loss: 0.6037
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.7500 - loss: 0.6551 - val_accuracy: 0.8438 - val_loss: 0.5335
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 661ms/step - accuracy: 0.7968 - loss: 0.5917 - val_accuracy: 0.8542 - val_loss: 0.5648
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.8125 - loss: 0.4319 - val_accuracy: 0.8333 - val_loss: 0.5759
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 659ms/step - accuracy: 0.8076 - loss: 0.5567 - val_accuracy: 0.8125 - val_

Model training finished.
Model saved to /home/qod120/Documents/project/2nd_ai_web_project/ai_model/cat_emotion_mobilenet_v2.h5
